# So Sanh Tong Hop 3 Mo Hinh: LBPH, FaceNet, ArcFace

Notebook so sanh hieu nang cua 3 mo hinh face recognition.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data tu 3 Models

In [ ]:
LOGS_BASE = Path('../logs')

with open(LOGS_BASE / 'arcface/training_history.json', 'r') as f:
    arcface_train = json.load(f)
with open(LOGS_BASE / 'arcface/evaluation_report_v2.json', 'r') as f:
    arcface_eval = json.load(f)

with open(LOGS_BASE / 'facenet/training_history.json', 'r') as f:
    facenet_train = json.load(f)
with open(LOGS_BASE / 'facenet/facenet_evaluation_report.json', 'r') as f:
    facenet_eval = json.load(f)

with open(LOGS_BASE / 'LBHP/metadata.json', 'r') as f:
    lbph_meta = json.load(f)

print('Data loaded successfully!')

## Bang So Sanh Metrics

In [ ]:
comparison_data = {
    'Model': ['LBPH', 'FaceNet', 'ArcFace'],
    'Type': ['Traditional ML', 'Deep Learning', 'Deep Learning'],
    'Embedding Size': ['-', 128, 512],
    'Top-1 Accuracy (%)': [lbph_meta['test_accuracy']*100, facenet_eval['metrics']['top1_accuracy'], arcface_eval['metrics']['top1_accuracy']],
    'Top-5 Accuracy (%)': ['-', facenet_eval['metrics']['top5_accuracy'], arcface_eval['metrics']['top5_accuracy']],
    'AUC-ROC': ['-', facenet_eval['metrics']['auc'], arcface_eval['metrics']['auc']],
    'EER (%)': ['-', facenet_eval['metrics']['eer']*100, arcface_eval['metrics']['eer']*100]
}
df = pd.DataFrame(comparison_data)
display(df)

## Bieu Do So Sanh

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

models = ['LBPH', 'FaceNet', 'ArcFace']
top1_acc = [lbph_meta['test_accuracy']*100, facenet_eval['metrics']['top1_accuracy'], arcface_eval['metrics']['top1_accuracy']]
colors = ['#95a5a6', '#e74c3c', '#2ecc71']

bars = axes[0].bar(models, top1_acc, color=colors, edgecolor='black')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Top-1 Accuracy Comparison')
axes[0].set_ylim([0, 100])
for bar, val in zip(bars, top1_acc):
    axes[0].text(bar.get_x()+bar.get_width()/2, val+2, f'{val:.1f}%', ha='center', fontweight='bold')

deep_models = ['FaceNet', 'ArcFace']
auc_vals = [facenet_eval['metrics']['auc'], arcface_eval['metrics']['auc']]
eer_vals = [facenet_eval['metrics']['eer']*100, arcface_eval['metrics']['eer']*100]

x = np.arange(len(deep_models))
width = 0.35
bars1 = axes[1].bar(x - width/2, auc_vals, width, label='AUC', color='#3498db')
bars2 = axes[1].bar(x + width/2, [v/100 for v in eer_vals], width, label='EER', color='#e74c3c')
axes[1].set_ylabel('Score')
axes[1].set_title('AUC vs EER (Deep Learning Models)')
axes[1].set_xticks(x)
axes[1].set_xticklabels(deep_models)
axes[1].legend()
axes[1].set_ylim([0, 1])

facenet_epochs = list(range(1, len(facenet_train['train_loss']) + 1))
arcface_epochs = list(range(1, len(arcface_train['history']['train_loss']) + 1))
axes[2].plot(facenet_epochs, [x*100 for x in facenet_train['val_acc']], 'r-', lw=2, label='FaceNet')
axes[2].plot(arcface_epochs, arcface_train['history']['val_acc'], 'g-', lw=2, label='ArcFace')
axes[2].axhline(y=lbph_meta['test_accuracy']*100, color='gray', linestyle='--', label='LBPH')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Validation Accuracy (%)')
axes[2].set_title('Training Progress Comparison')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(LOGS_BASE / 'model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## Radar Chart So Sanh

In [ ]:
categories = ['Top-1 Acc', 'Top-5 Acc', 'AUC', '1-EER', 'Speed']

lbph_vals = [lbph_meta['test_accuracy'], 0.1, 0.5, 0.5, 1.0]
facenet_vals = [facenet_eval['metrics']['top1_accuracy']/100, facenet_eval['metrics']['top5_accuracy']/100, 
                facenet_eval['metrics']['auc'], 1-facenet_eval['metrics']['eer'], 0.8]
arcface_vals = [arcface_eval['metrics']['top1_accuracy']/100, arcface_eval['metrics']['top5_accuracy']/100,
                arcface_eval['metrics']['auc'], 1-arcface_eval['metrics']['eer'], 0.6]

angles = [n / float(len(categories)) * 2 * np.pi for n in range(len(categories))]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for vals, name, color in [(lbph_vals, 'LBPH', '#95a5a6'), (facenet_vals, 'FaceNet', '#e74c3c'), (arcface_vals, 'ArcFace', '#2ecc71')]:
    vals = vals + vals[:1]
    ax.plot(angles, vals, 'o-', linewidth=2, label=name, color=color)
    ax.fill(angles, vals, alpha=0.1, color=color)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax.set_title('Model Performance Comparison Radar', pad=20)

plt.tight_layout()
plt.savefig(LOGS_BASE / 'radar_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## Bao Cao Tong Hop

In [ ]:
report = f'''
# BAO CAO SO SANH 3 MO HINH FACE RECOGNITION

## 1. TONG QUAN

| Model | Type | Embedding | Top-1 Acc | AUC | EER |
|-------|------|-----------|-----------|-----|-----|
| LBPH | Traditional | - | {lbph_meta["test_accuracy"]*100:.2f}% | - | - |
| FaceNet | Deep Learning | 128 | {facenet_eval["metrics"]["top1_accuracy"]:.2f}% | {facenet_eval["metrics"]["auc"]:.4f} | {facenet_eval["metrics"]["eer"]*100:.2f}% |
| ArcFace | Deep Learning | 512 | {arcface_eval["metrics"]["top1_accuracy"]:.2f}% | {arcface_eval["metrics"]["auc"]:.4f} | {arcface_eval["metrics"]["eer"]*100:.2f}% |

## 2. NHAN XET

### ArcFace (TOT NHAT)
- Top-1 Accuracy cao nhat: {arcface_eval["metrics"]["top1_accuracy"]:.2f}%
- AUC cao nhat: {arcface_eval["metrics"]["auc"]:.4f}
- EER thap nhat: {arcface_eval["metrics"]["eer"]*100:.2f}%

### FaceNet
- Top-1 Accuracy: {facenet_eval["metrics"]["top1_accuracy"]:.2f}%
- Embedding nho hon (128 vs 512) -> nhanh hon
- EER cao hon ArcFace: {facenet_eval["metrics"]["eer"]*100:.2f}%

### LBPH
- Accuracy rat thap: {lbph_meta["test_accuracy"]*100:.2f}%
- Khong phu hop voi large-scale recognition
- Phu hop cho small-scale applications

## 3. KET LUAN

**ArcFace** la mo hinh tot nhat cho bai toan face recognition voi:
- Hieu suat cao nhat tren tat ca metrics
- Phu hop cho production systems
- Trade-off: Yeu cau GPU va thoi gian training lon hon
'''
display(Markdown(report))

In [ ]:
summary = {
    'comparison_date': '2025-12-20',
    'models': {
        'lbph': {'type': 'Traditional', 'top1': lbph_meta['test_accuracy']*100},
        'facenet': {'type': 'Deep Learning', 'embedding': 128, 'top1': facenet_eval['metrics']['top1_accuracy'], 'auc': facenet_eval['metrics']['auc'], 'eer': facenet_eval['metrics']['eer']},
        'arcface': {'type': 'Deep Learning', 'embedding': 512, 'top1': arcface_eval['metrics']['top1_accuracy'], 'auc': arcface_eval['metrics']['auc'], 'eer': arcface_eval['metrics']['eer']}
    },
    'best_model': 'ArcFace'
}
with open(LOGS_BASE / 'comparison_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print('Summary saved!')
print(json.dumps(summary, indent=2))